In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
import kagglehub

# Download latest version
path = kagglehub.model_download("google/paligemma-2/transformers/paligemma2-3b-pt-224")

print("Path to model files:", path)

Path to model files: /kaggle/input/paligemma-2/transformers/paligemma2-3b-pt-224/1


In [3]:
# Install dependencies
!pip install -q transformers peft nltk rouge-score wandb

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 

In [4]:
# Verify GPU
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.5.1+cu124
True


In [8]:
import wandb
wandb.login(key="d070aabfe54f4733fb727662604b037dee34842c")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory

data = pd.read_csv("/kaggle/input/riscmm/RISCM/captions.csv")
data.head()

,source,split,image,caption_1,caption_2,caption_3,caption_4,caption_5
0,NWPU,test,NWPU_31430.jpg,A gray plane on the runway and the lawn beside .,A grey plane is on the runway by the lawn .,There is an airplane on the runway with a larg...,A plane is parked on the runway next to the gr...,There is a plane on the runway beside the grass .
1,NWPU,test,NWPU_31431.jpg,Three small planes parked in a line on the air...,"There are four aircraft on the open ground, Th...",There are many planes of different sizes in a ...,Four planes are parked on the runway .,Four planes of different sizes were on the mar...
2,NWPU,test,NWPU_31432.jpg,A plane parked in a line on the airport with s...,A white plane was parked on the instruction li...,An airplane parked in an open area with many c...,A plane is parked on the open space .,There is 1 plane on the ground marked .
3,NWPU,test,NWPU_31433.jpg,A small plane and a big plane parked next to b...,A white plane and a gray plane parked at the b...,Two planes of different sizes are neatly parke...,A large plane and a small plane are parked nea...,Two planes are on the marked ground .
4,NWPU,test,NWPU_31434.jpg,Two planes parked next to boarding bridges .,Two aircraft were parked at the departure gates .,Two planes of different sizes are neatly parke...,Two planes are parked next to the terminal .,Two planes are on the marked ground .


Load Dataset with Small Partition

In [6]:
import pandas as pd
import os

def load_small_partition(image_dir, caption_file, sample_size=200):
    df = pd.read_csv(caption_file)

    # Filter out missing images
    valid_images = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
    df = df[df['image'].isin(valid_images)]

    # Use existing splits
    train_df = df[df['split'] == 'train'].sample(frac=1, random_state=42).head(sample_size)
    val_df = df[df['split'] == 'test'].sample(frac=1, random_state=42).head(int(0.2 * sample_size))

    print(f"Loaded small partition: {len(train_df)} train, {len(val_df)} val")
    return train_df.reset_index(drop=True), val_df.reset_index(drop=True)

# Set paths
image_dir = "/kaggle/input/riscmm/RISCM/resized"
caption_file = "/kaggle/input/riscmm/RISCM/captions.csv"

# Load data
train_df, val_df = load_small_partition(image_dir, caption_file, sample_size=200)

Loaded small partition: 200 train, 40 val


Training Function

In [7]:
import torch
from transformers import PaliGemmaForConditionalGeneration, PaliGemmaProcessor
from peft import LoraConfig, get_peft_model
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
import torch.optim as optim

class RISCDataset(Dataset):
    def __init__(self, image_dir, df):
        self.image_dir = image_dir
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row.image)
        image = Image.open(image_path).convert('RGB')
        caption = row.caption_1
        return {"image": image, "caption": caption}

def custom_collate_fn(batch):
    images = [item["image"] for item in batch]
    captions = [item["caption"] for item in batch]
    return {"images": images, "captions": captions}

def train_lora(model_name, image_dir, train_df, val_df, caption_file, output_dir,
               lora_rank=4, epochs=3, learning_rate=1e-4,
               max_train_samples=None, max_val_samples=None,
               batch_size=1, accum_steps=8,
               target_modules=["q_proj", "v_proj"]):

    wandb.init(project="DI725_Phase2", name=f"LoRA-R{lora_rank}")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = PaliGemmaForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
    processor = PaliGemmaProcessor.from_pretrained(model_name, use_fast=True)

    lora_config = LoraConfig(
        r=lora_rank,
        lora_alpha=32,
        target_modules=target_modules,
        lora_dropout=0.1
    )
    model = get_peft_model(model, lora_config)

    # Use provided train/val DataFrames
    if max_train_samples:
        train_df = train_df.head(max_train_samples)
    if max_val_samples:
        val_df = val_df.head(max_val_samples)

    train_dataset = RISCDataset(image_dir, train_df)
    val_dataset = RISCDataset(image_dir, val_df)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    scaler = GradScaler()

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        steps = 0
        optimizer.zero_grad()

        for batch_idx, batch in enumerate(train_loader):
            try:
                images = batch["images"]
                captions = [f"<image> caption {cap}" for cap in batch["captions"]]

                inputs = processor(text=captions, images=images, return_tensors="pt", padding="longest").to(device)

                with autocast("cuda"):
                    outputs = model(
                        input_ids=inputs["input_ids"],
                        attention_mask=inputs["attention_mask"],
                        pixel_values=inputs["pixel_values"],
                        labels=inputs["input_ids"]
                    )
                    loss = outputs.loss / accum_steps

                scaler.scale(loss).backward()

                if (batch_idx + 1) % accum_steps == 0 or (batch_idx + 1) == len(train_loader):
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()

                total_loss += loss.item() * accum_steps
                steps += 1

                if steps % 50 == 0:
                    print(f"Epoch {epoch+1}, Step {steps}, Loss: {loss.item() * accum_steps:.4f}")

            except Exception as e:
                print(f"Error in batch {batch_idx}: {e}")
                continue

        avg_train_loss = total_loss / steps if steps > 0 else 0
        wandb.log({"epoch": epoch+1, "train_loss": avg_train_loss})

        # Validation loop
        model.eval()
        val_loss = 0
        val_steps = 0
        for batch in val_loader:
            try:
                images = batch["images"]
                captions = [f"<image> caption {cap}" for cap in batch["captions"]]
                inputs = processor(text=captions, images=images, return_tensors="pt", padding="longest").to(device)

                with torch.no_grad(), autocast("cuda"):
                    outputs = model(**inputs, labels=inputs["input_ids"])
                    val_loss += outputs.loss.item()
                val_steps += 1

            except Exception as e:
                print(f"Validation error: {e}")
                continue

        avg_val_loss = val_loss / val_steps if val_steps > 0 else 0
        wandb.log({"epoch": epoch+1, "val_loss": avg_val_loss})
        print(f"Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}")
        model.train()

    model.save_pretrained(output_dir)
    processor.save_pretrained(output_dir)
    wandb.finish()

2025-05-16 13:40:22.387653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747402822.591277      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747402822.647998      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Evaluation Functions

In [9]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from transformers import PaliGemmaForConditionalGeneration, PaliGemmaProcessor
from peft import PeftModel 
from PIL import Image
import torch
from torch.amp import autocast
import pandas as pd
import os

# Define smoother for BLEU
smoothie = SmoothingFunction().method1

def evaluate_zero_shot(model_name, image_dir, val_df, num_samples=50):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = PaliGemmaForConditionalGeneration.from_pretrained(model_name).to(device)
    processor = PaliGemmaProcessor.from_pretrained(model_name, use_fast=True)
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    bleu_scores = []
    rouge_scores = []

    print("Running zero-shot evaluation...")
    val_df = val_df.reset_index(drop=True)

    for i in range(min(num_samples, len(val_df))):
        try:
            row = val_df.iloc[i]
            image_path = os.path.join(image_dir, row.image)
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Image not found: {image_path}")
            image = Image.open(image_path).convert('RGB')
            prompt = "<image> caption"

            inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

            with torch.no_grad(), autocast("cuda"):
                output_ids = model.generate(**inputs, max_new_tokens=50)
                caption = processor.decode(output_ids[0], skip_special_tokens=True)

            reference = row.caption_1
            hypothesis = caption

            # BLEU with smoothing
            bleu = sentence_bleu([reference.split()], hypothesis.split(), smoothing_function=smoothie)
            bleu_scores.append(bleu)

            # ROUGE-L
            rs = scorer.score(reference, hypothesis)['rougeL'].fmeasure
            rouge_scores.append(rs)

            print(f"\nReference: {reference}\nHypothesis: {hypothesis}")

        except Exception as e:
            print(f"Error evaluating sample {i}: {e}")
            continue

    avg_bleu = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    avg_rouge = sum(rouge_scores) / len(rouge_scores) if rouge_scores else 0

    print(f"\nZero-shot Results: BLEU-4={avg_bleu:.4f}, ROUGE-L={avg_rouge:.4f}")

    return {
        "zero_shot_BLEU": avg_bleu,
        "zero_shot_ROUGE_L": avg_rouge
    }


def evaluate_lora_model(lora_model_path, base_model_name, image_dir, val_df, num_samples=50):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    base_model = PaliGemmaForConditionalGeneration.from_pretrained(base_model_name).to(device)
    model = PeftModel.from_pretrained(base_model, lora_model_path).to(device)
    processor = PaliGemmaProcessor.from_pretrained(base_model_name)
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    bleu_scores = []
    rouge_scores = []

    print("Running LoRA model evaluation...")
    val_df = val_df.reset_index(drop=True)

    for i in range(min(num_samples, len(val_df))):
        try:
            row = val_df.iloc[i]
            image_path = os.path.join(image_dir, row.image)
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Image not found: {image_path}")
            image = Image.open(image_path).convert('RGB')
            prompt = "<image> caption"

            inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

            with torch.no_grad(), autocast("cuda"):
                output_ids = model.generate(**inputs, max_new_tokens=50)
                caption = processor.decode(output_ids[0], skip_special_tokens=True)

            reference = row.caption_1
            hypothesis = caption

            # BLEU with smoothing
            bleu = sentence_bleu([reference.split()], hypothesis.split(), smoothing_function=smoothie)
            bleu_scores.append(bleu)

            # ROUGE-L
            rs = scorer.score(reference, hypothesis)['rougeL'].fmeasure
            rouge_scores.append(rs)

            print(f"\nReference: {reference}\nHypothesis: {hypothesis}")

        except Exception as e:
            print(f"Error evaluating sample {i}: {e}")
            continue

    avg_bleu = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    avg_rouge = sum(rouge_scores) / len(rouge_scores) if rouge_scores else 0

    print(f"\nLoRA Model Results: BLEU-4={avg_bleu:.4f}, ROUGE-L={avg_rouge:.4f}")

    return {
        "lora_BLEU": avg_bleu,
        "lora_ROUGE_L": avg_rouge
    }

Inference Visualization

In [10]:
import matplotlib.pyplot as plt

def visualize_inference(model, processor, val_df, image_dir, num_samples=5):
    fig, axes = plt.subplots(num_samples, 1, figsize=(10, 5 * num_samples))
    val_df = val_df.reset_index(drop=True)

    for i in range(min(num_samples, len(val_df))):
        row = val_df.iloc[i]
        image_path = os.path.join(image_dir, row.image)
        image = Image.open(image_path).convert('RGB')
        prompt = "<image> caption"

        inputs = processor(text=prompt, images=image, return_tensors="pt").to("cuda")
        with torch.no_grad(), autocast("cuda"):
            output_ids = model.generate(**inputs, max_new_tokens=50)
            caption = processor.decode(output_ids[0], skip_special_tokens=True)

        ax = axes[i] if num_samples > 1 else axes
        ax.imshow(image)
        ax.axis("off")
        ax.set_title(f"True: {row.caption_1}\nPred: {caption}", fontsize=10)

    plt.tight_layout()
    plt.savefig("generated_captions.png", dpi=300)
    plt.show()

    # Log to WandB
    wandb.log({"Generated Captions": [wandb.Image("generated_captions.png")]})

Train Multiple LoRA Configurations

In [11]:
import wandb
wandb.login()

# Set environment variable for better memory management
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Define evaluation sample size
num_samples = 10  # ← Define here to avoid NameError

configs = [
    {"lora_rank": 4, "name": "LoRA-R4"},
    {"lora_rank": 8, "name": "LoRA-R8"},
    {"lora_rank": 4, "name": "LoRA-R4-k-o", "target_modules": ["k_proj", "o_proj"]}
]

results = []

# Optional: Evaluate Zero-shot Baseline First
wandb.init(project="DI725_Phase2", name="ZeroShot-Baseline")
zero_shot_results = evaluate_zero_shot(
    model_name="/kaggle/input/paligemma-2/transformers/paligemma2-3b-pt-224/1",
    image_dir=image_dir,
    val_df=val_df,
    num_samples=num_samples  # ← Now defined
)
wandb.log({
    "zero_shot_BLEU": zero_shot_results["zero_shot_BLEU"],
    "zero_shot_ROUGE_L": zero_shot_results["zero_shot_ROUGE_L"]
})
wandb.finish()

# Train and evaluate each LoRA config
for config in configs:
    print(f"\n🚀 Training: {config['name']}")
    output_dir = f"./{config['name']}"

    wandb.init(
        project="DI725_Phase2",
        name=config["name"],
        config={
            "lora_rank": config.get("lora_rank"),
            "target_modules": "-".join(config.get("target_modules", ["q_proj", "v_proj"])),
            "num_samples_eval": num_samples
        },
        reinit=True
    )

    # Run training
    train_lora(
        model_name="/kaggle/input/paligemma-2/transformers/paligemma2-3b-pt-224/1",
        image_dir=image_dir,
        caption_file=caption_file,
        train_df=train_df,
        val_df=val_df,
        output_dir=output_dir,
        lora_rank=config.get("lora_rank", 4),
        target_modules=config.get("target_modules", ["q_proj", "v_proj"]),
        epochs=1,
        learning_rate=1e-4,
        batch_size=1,
        accum_steps=8
    )

    # Run evaluation
    result = evaluate_lora_model(
        output_dir,
        "/kaggle/input/paligemma-2/transformers/paligemma2-3b-pt-224/1",
        image_dir,
        val_df,
        num_samples=num_samples  # Pass consistent value
    )
    result["config"] = config["name"]
    result["rank"] = config.get("lora_rank", 4)
    result["modules"] = "-".join(config.get("target_modules", ["q_proj", "v_proj"]))
    results.append(result)

    # Log metrics
    wandb.log({
        "lora_BLEU": result["lora_BLEU"],
        "lora_ROUGE_L": result["lora_ROUGE_L"]
    })
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Running zero-shot evaluation...
Error evaluating sample 0: CUDA out of memory. Tried to allocate 42.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 45.12 MiB is free. Process 4454 has 15.84 GiB memory in use. Of the allocated memory 15.51 GiB is allocated by PyTorch, and 46.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Error evaluating sample 1: CUDA out of memory. Tried to allocate 42.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 23.12 MiB is free. Process 4454 has 15.86 GiB memory in use. Of the allocated memory 15.51 GiB is allocated by PyTorch, and 66.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation

zero_shot_BLEU,▁
zero_shot_ROUGE_L,▁
zero_shot_BLEU,0
zero_shot_ROUGE_L,0



🚀 Training: LoRA-R4


...

epoch,▁▁▅▅██
train_loss,█▂▁
val_loss,█▃▁
epoch,3
train_loss,12.06554
val_loss,11.96967


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Running LoRA model evaluation...

Reference: The storage tanks here are half white and half black .
Hypothesis:  caption
A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A

Reference: The stratus clouds are located above the surface of the sea .
Hypothesis:  caption
A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A

Reference: The long strip island with dense vegetation is surrounded by light blue waters .
Hypothesis:  caption
A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A

Reference: The harbor has lots of neatly docked boats and some buildings are next to the harbor .
Hypothesis:  caption
ⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧ

Reference: parking lot next to planted a few trees .
Hypothesis:  caption
ⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧⓧ

Reference: A road go across another one diagonally .
Hypothesis:  caption
A A 

Error: You must call wandb.init() before wandb.log()

 Generate Table

In [ ]:
import pandas as pd

# Format results
results_table = pd.DataFrame(results)
results_table = results_table.rename(columns={
    "lora_BLEU": "BLEU-4",
    "lora_ROUGE_L": "ROUGE-L"
})[["config", "rank", "modules", "BLEU-4", "ROUGE-L"]]

# Add zero-shot baseline
zero_shot_row = {
    "config": "Zero-shot Baseline",
    "rank": "N/A",
    "modules": "N/A",
    "BLEU-4": f"{zero_shot_results['zero_shot_BLEU']:.4f}",
    "ROUGE-L": f"{zero_shot_results['zero_shot_ROUGE_L']:.4f}"
}

# Prepend zero-shot
results_table = pd.concat([pd.DataFrame([zero_shot_row]), results_table], ignore_index=True)

print("\n📊 Model Performance Comparison:")
print(results_table.to_markdown(index=False))

# Save to CSV
results_table.to_csv("results_table.csv", index=False)

# Upload to WandB
artifact = wandb.Artifact("results_table", type="evaluation")
artifact.add(wandb.Table(dataframe=results_table), "results_table")
wandb.init(project="DI725_Phase2", name="FinalResults")
wandb.log_artifact(artifact)
wandb.finish()

Visualize Sample Captions from Best Model

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plot_df = results_table[results_table["config"] != "Zero-shot Baseline"].copy()
plot_df["BLEU-4"] = plot_df["BLEU-4"].astype(float)
plot_df["ROUGE-L"] = plot_df["ROUGE-L"].astype(float)

plt.figure(figsize=(10, 6))
sns.barplot(data=plot_df, x="config", y="BLEU-4", label="BLEU-4", ci=None)
sns.barplot(data=plot_df, x="config", y="ROUGE-L", label="ROUGE-L", ci=None, alpha=0.6)

plt.xticks(rotation=45)
plt.title("Model Performance Comparison")
plt.ylabel("Score")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
best_model_path = "./LoRA-R8"
base_model_name = "/kaggle/input/paligemma-2/transformers/paligemma2-3b-pt-224/1"
processor = PaliGemmaProcessor.from_pretrained(base_model_name)
model = PaliGemmaForConditionalGeneration.from_pretrained(best_model_path).to("cuda")

visualize_inference(model, processor, val_df, image_dir, num_samples=5)